## Proof of Work (PoW) and Proof of Stake (PoS) consensus protocols in a blockchain simulation.

### Introduction

This program offers a comprehensive look into the basic principles of blockchain technology, such as the formation, authentication, and administration of blocks within a chain. The program encompasses both Proof of Work (PoW) and Proof of Stake (PoS) consensus methods, enabling users to experience different approaches to reaching an agreement within a distributed network.

The blockchain commences with the inception of a Genesis Block, which acts as the origin of the chain. Users are then prompted to determine if they wish to mine additional blocks, choosing a consensus method for each block. Under the PoW approach, miners solve computational puzzles by identifying a nonce value that meets specific difficulty requirements. This process, known as mining, rewards the successful miner with the ability to add the new block to the chain. In the PoS mechanism, validators are selected based on their stake, in other word, the cryptocurrency they hold, with a higher stake increasing the likelihood of being chosen as the validator for the next block.

As new blocks are added, the program continually verifies the integrity of the blockchain. It ensures that each block's hash is accurately calculated based on its contents and the previous block's hash, guaranteeing that the chain is unaltered and has not been tampered with. Additionally, specific checks are performed for PoW blocks (validating if the hash meets the difficulty criteria) and PoS blocks (confirming the inclusion of the validator information).

Throughout the process, the program generates detailed output, displaying the specifics of each newly created block, including its hash, timestamp, data, previous hash, and nonce value. Users can continue mining new blocks or exit the program, at which point the program checks and reports the validity of the entire blockchain.

This design serves as a hands-on illustration of the fundamental concepts behind blockchain technology, enabling users to understand the concept of block formation, consensus methods, and chain verification.

In [2]:
import hashlib  # Importing the hashlib library to help with creating unique hashes with sha256
import time  # Importing the time library to get the current time
from datetime import datetime  # Importing the datetime library to format the time
import random  # Importing the random library to help with selecting a stakeholder in Proof of Stake

# This class represents a block in the blockchain
class Block:
    # Defining instance for the Block class
    def __init__(self, index, previous_hash, timestamp, data, hash, nonce=0):
        self.index = index  # The position of the block in the chain
        self.previous_hash = previous_hash  # The unique identifier of the previous block
        self.timestamp = timestamp  # The time when the block was created
        self.data = data  # The information stored in the block
        self.hash = hash  # The unique identifier of the block
        self.nonce = nonce  # A number used in the Proof of Work process

# This class represents the entire blockchain
class Blockchain:
    # Defining instance for the Blockchain class
    def __init__(self):
        self.chain = []  # A list to store all blocks in the blockchain
        self.difficulty = 2  # The difficulty level for creating a new block (used in Proof of Work)
        self.stakeholders = {"Esther": 30, "David": 75, "Abiskar": 60}  # People who have a stake in the blockchain (used in Proof of Stake)
        self.consensus = None  # To keep track of the selected consensus mechanism (PoW or PoS)

    # This function creates the first block in the blockchain
    def create_genesis_block(self):
        print("Creating Genesis Block...")
        timestamp = time.time()  # Get the current time
        genesis_block = Block(0, "0", timestamp, "Genesis Block", self.calculate_hash(0, "0", timestamp, "Genesis Block", 0))
        print("Genesis Block created.")
        return genesis_block

    # This function calculates hash for a block
    def calculate_hash(self, index, previous_hash, timestamp, data, nonce):
        value = str(index) + str(previous_hash) + str(timestamp) + str(data) + str(nonce)
        return hashlib.sha256(value.encode('utf-8')).hexdigest() # Performing hashing process with sha256

    # This function finds a valid nonce to create a new block for Proof of Work
    def proof_of_work(self, index, previous_hash, timestamp, data):
        nonce = 0
        while True:
            hash = self.calculate_hash(index, previous_hash, timestamp, data, nonce)
            if hash[:self.difficulty] == '0' * self.difficulty:
                return nonce, hash
            nonce += 1

    # This function selects a stakeholder based on their stake in Proof of Stake
    def proof_of_stake(self):
        total_stake = sum(self.stakeholders.values())  # Total amount of stakes
        pick = random.uniform(0, total_stake)  # Randomly select an amount
        current = 0  # Set the current amount to initial state of 0
        for stakeholder, stake in self.stakeholders.items():
            current += stake  # Add stake to the current staked amount
            if current >= pick:  # Pick the stakeholder with highest stake 
                return stakeholder

    # This function adds a new block to the blockchain
    def add_block(self, data, previous_hash):
        index = len(self.chain)  # Get the current position in the chain
        timestamp = time.time()  # Get the current time
        if self.consensus == "pow":  # If using Proof of Work
            nonce, hash = self.proof_of_work(index, previous_hash, timestamp, data)
        elif self.consensus == "pos":  # If using Proof of Stake
            validator = self.proof_of_stake()
            nonce = 0  # Nonce is not used in Proof of Stake
            data = f"{data} (Validator: {validator})"
            hash = self.calculate_hash(index, previous_hash, timestamp, data, nonce)
        new_block = Block(index, previous_hash, timestamp, data, hash, nonce)
        self.chain.append(new_block)  # Add the new block to the chain
        print(f"Block #{index} created using {self.consensus.upper()}.")
        self.print_block_details(new_block)

    # This function checks if the blockchain is valid
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            # Validate block hash
            expected_hash = self.calculate_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data, current_block.nonce)
            if current_block.hash != expected_hash:
                print(f"Invalid hash at block {current_block.index}")
                print(f"Expected hash: {expected_hash}, but got: {current_block.hash}")
                return False

            # Validate previous hash
            if current_block.previous_hash != previous_block.hash:
                print(f"Invalid previous hash at block {current_block.index}")
                print(f"Expected previous hash: {previous_block.hash}, but got: {current_block.previous_hash}")
                return False

            # Additional PoW validation
            if self.consensus == "pow" and current_block.hash[:self.difficulty] != '0' * self.difficulty:
                print(f"PoW validation failed at block {current_block.index}")
                return False

            # Additional PoS validation
            if self.consensus == "pos" and "(Validator: " not in current_block.data:
                print(f"PoS validation failed at block {current_block.index}")
                print(f"Validator information missing in block data: {current_block.data}")
                return False

        return True

    # This function prints the details of a block
    def print_block_details(self, block):
        timestamp = datetime.fromtimestamp(block.timestamp).strftime('%Y-%m-%d %H:%M:%S')
        print(f"Printing Block #{block.index}...")
        print(f"Hash: {block.hash}\nTimestamp: {timestamp}\nData: {block.data}\nPrevious Hash: {block.previous_hash}\nNonce: {block.nonce}\n")

# Test the blockchain
blockchain = Blockchain()

# Create genesis block
genesis_block = blockchain.create_genesis_block()
blockchain.chain.append(genesis_block)

# Print genesis block details
blockchain.print_block_details(genesis_block)

# Prompt for mining new blocks

new_block_mined = False  # Flag to track if a new block has been mined

while True:
    response = input("Do you want to mine a new block? (y/n): ")
    if response.lower() == 'y':  # If user wants to mine a new block
        if blockchain.consensus is None:  # If consensus mechanism is not yet set
            consensus = input("Enter the consensus mechanism (pow/pos): ").lower()
            if consensus not in ["pow", "pos"]:
                print("Invalid input. Please enter 'pow' or 'pos'.\n")
                continue
            blockchain.consensus = consensus

        genesis_hash_input = input("Enter the Genesis Block hash to create a new block: ")
        if genesis_hash_input == blockchain.chain[0].hash:  # Check if the input hash matches the genesis block hash
            blockchain.add_block("Transaction Data", blockchain.chain[-1].hash)
            new_block_mined = True  # Update the flag when a new block is mined
        else:
            print("Invalid Genesis Block hash. Block not created.\n")
    elif response.lower() == 'n':  # If user does not want to mine a new block
        if blockchain.is_chain_valid():  # Validate the blockchain
            print("Blockchain is valid.")
        else:
            print("Blockchain is not valid.")
        blockchain.consensus = None  # Reset consensus mechanism for new process
        break
    else:
        print("Invalid input. Please enter 'y' or 'n'.\n")

Creating Genesis Block...
Genesis Block created.
Printing Block #0...
Hash: a23f1109c8fd43d6a7b3cc299a2c510b0802c626b356954f67377581a53fad85
Timestamp: 2024-05-30 07:16:55
Data: Genesis Block
Previous Hash: 0
Nonce: 0

Do you want to mine a new block? (y/n): y
Enter the consensus mechanism (pow/pos): pow
Enter the Genesis Block hash to create a new block: a23f1109c8fd43d6a7b3cc299a2c510b0802c626b356954f67377581a53fad85
Block #1 created using POW.
Printing Block #1...
Hash: 005c5e90a090bb6408ccb4fdcd57e8894c56ba2be0b51e63a0b67d668c1e1b89
Timestamp: 2024-05-30 07:20:36
Data: Transaction Data
Previous Hash: a23f1109c8fd43d6a7b3cc299a2c510b0802c626b356954f67377581a53fad85
Nonce: 906

Do you want to mine a new block? (y/n): y
Enter the Genesis Block hash to create a new block: a23f1109c8fd43d6a7b3cc299a2c510b0802c626b356954f67377581a53fad85
Block #2 created using POW.
Printing Block #2...
Hash: 007a7d67ccfa8359c9fb1aa739fee20f85d087229ff5a8e8da712d9884e3cbb1
Timestamp: 2024-05-30 07:20:41
D